# LSASS Memory Dump
Detects process LSASS memory dump using procdump or taskmgr based on the CallTrace pointing to dbghelp.dll or dbgcore.dll for win10

## Rule Content
```
- title: LSASS Memory Dump
  id: 5ef9853e-4d0e-4a70-846f-a9ca37d876da
  status: experimental
  description: Detects process LSASS memory dump using procdump or taskmgr based on
    the CallTrace pointing to dbghelp.dll or dbgcore.dll for win10
  author: Samir Bousseaden
  references:
  - https://blog.menasec.net/2019/02/threat-hunting-21-procdump-or-taskmgr.html
  tags:
  - attack.t1003
  - attack.s0002
  - attack.credential_access
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 10
      TargetImage: C:\windows\system32\lsass.exe
      GrantedAccess: '0x1fffff'
      CallTrace:
      - '*dbghelp.dll*'
      - '*dbgcore.dll*'
    condition: selection
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"10" AND target_process_path:"C\:\\windows\\system32\\lsass.exe" AND process_granted_access:"0x1fffff" AND process_call_trace.keyword:(*dbghelp.dll* OR *dbgcore.dll*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()